In [122]:
%pip install polars seaborn


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [123]:
import polars as pl

option_df = pl.read_parquet("test_data/options_minute_aggs.parquet")

print(option_df.head())

shape: (5, 8)
┌────────────────────┬────────┬───────┬───────┬───────┬───────┬─────────────────────┬──────────────┐
│ ticker             ┆ volume ┆ open  ┆ close ┆ high  ┆ low   ┆ window_start        ┆ transactions │
│ ---                ┆ ---    ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---                 ┆ ---          │
│ str                ┆ i64    ┆ f64   ┆ f64   ┆ f64   ┆ f64   ┆ i64                 ┆ i64          │
╞════════════════════╪════════╪═══════╪═══════╪═══════╪═══════╪═════════════════════╪══════════════╡
│ O:A240621C00110000 ┆ 4      ┆ 22.11 ┆ 22.11 ┆ 22.11 ┆ 22.11 ┆ 1718115060000000000 ┆ 1            │
│ O:A240621C00135000 ┆ 1      ┆ 1.0   ┆ 1.0   ┆ 1.0   ┆ 1.0   ┆ 1718112600000000000 ┆ 1            │
│ O:A240621C00135000 ┆ 1      ┆ 0.85  ┆ 0.85  ┆ 0.85  ┆ 0.85  ┆ 1718113140000000000 ┆ 1            │
│ O:A240621C00135000 ┆ 1      ┆ 0.87  ┆ 0.87  ┆ 0.87  ┆ 0.87  ┆ 1718113200000000000 ┆ 1            │
│ O:A240621C00135000 ┆ 1      ┆ 0.85  ┆ 0.85  ┆ 0.85  ┆ 0.85  ┆ 1718115060000

In [124]:
option_df = option_df.select('ticker', 'volume', 'open', 'window_start')
option_df = option_df.rename({'ticker': 'option_ticker', 'open': 'option_price', 'window_start': 'timestamp'})
option_df.head()

option_ticker,volume,option_price,timestamp
str,i64,f64,i64
"""O:A240621C00110000""",4,22.11,1718115060000000000
"""O:A240621C00135000""",1,1.0,1718112600000000000
"""O:A240621C00135000""",1,0.85,1718113140000000000
"""O:A240621C00135000""",1,0.87,1718113200000000000
"""O:A240621C00135000""",1,0.85,1718115060000000000


In [125]:
# Parse Polygon option tickers to useful columns
regex_pattern = r"O:([A-Za-z]+)([0-9]+)([PC])([0-9]+)"
option_df = option_df.with_columns(
    pl.col('option_ticker').str.extract(regex_pattern, 1).alias('underlying_ticker'),
    pl.col('option_ticker').str.extract(regex_pattern, 2).alias('expiration_date'),
    pl.col('option_ticker').str.extract(regex_pattern, 3).alias('option_type'),
    pl.col('option_ticker').str.extract(regex_pattern, 4).alias('strike_price'),
)
option_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,f64,i64,str,str,str,str
"""O:A240621C00110000""",4,22.11,1718115060000000000,"""A""","""240621""","""C""","""00110000"""
"""O:A240621C00135000""",1,1.0,1718112600000000000,"""A""","""240621""","""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718113140000000000,"""A""","""240621""","""C""","""00135000"""
"""O:A240621C00135000""",1,0.87,1718113200000000000,"""A""","""240621""","""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718115060000000000,"""A""","""240621""","""C""","""00135000"""


In [126]:
# Expiration date can sometimes include an extra int at the front (because Polygon idk) so we need to filter those out and transform to proper type
option_df = option_df.with_columns(
    pl.when(pl.col('expiration_date').str.len_chars() == 7)
    .then(pl.col("expiration_date").str.slice(1))
    .otherwise(pl.col('expiration_date'))
    .alias('expiration_date')
)

option_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,f64,i64,str,str,str,str
"""O:A240621C00110000""",4,22.11,1718115060000000000,"""A""","""240621""","""C""","""00110000"""
"""O:A240621C00135000""",1,1.0,1718112600000000000,"""A""","""240621""","""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718113140000000000,"""A""","""240621""","""C""","""00135000"""
"""O:A240621C00135000""",1,0.87,1718113200000000000,"""A""","""240621""","""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718115060000000000,"""A""","""240621""","""C""","""00135000"""


In [127]:
# Need to expand the expiration_date to be able to transform to datetime
option_df = option_df.with_columns(
    pl.col("expiration_date").map_elements(lambda date_str: f"20{date_str[0:2]}-{date_str[2:4]}-{date_str[4:]}", return_dtype=pl.self_dtype()).alias("expiration_date")
)

option_df.head()   

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,f64,i64,str,str,str,str
"""O:A240621C00110000""",4,22.11,1718115060000000000,"""A""","""2024-06-21""","""C""","""00110000"""
"""O:A240621C00135000""",1,1.0,1718112600000000000,"""A""","""2024-06-21""","""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718113140000000000,"""A""","""2024-06-21""","""C""","""00135000"""
"""O:A240621C00135000""",1,0.87,1718113200000000000,"""A""","""2024-06-21""","""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718115060000000000,"""A""","""2024-06-21""","""C""","""00135000"""


In [128]:
# Convert the expiration date to date to make it easier to work with
option_df = option_df.with_columns(
    pl.col("expiration_date").str.to_datetime(time_unit="ns")
)

option_df.head()  

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,f64,i64,str,datetime[ns],str,str
"""O:A240621C00110000""",4,22.11,1718115060000000000,"""A""",2024-06-21 00:00:00,"""C""","""00110000"""
"""O:A240621C00135000""",1,1.0,1718112600000000000,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718113140000000000,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""
"""O:A240621C00135000""",1,0.87,1718113200000000000,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718115060000000000,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""


In [129]:
# Convert option price to decimal
option_df = option_df.with_columns(
    pl.col('option_price').cast(pl.Decimal(scale=2)).alias('option_price')
)

option_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,"decimal[*,2]",i64,str,datetime[ns],str,str
"""O:A240621C00110000""",4,22.11,1718115060000000000,"""A""",2024-06-21 00:00:00,"""C""","""00110000"""
"""O:A240621C00135000""",1,1.00,1718112600000000000,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718113140000000000,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""
"""O:A240621C00135000""",1,0.87,1718113200000000000,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,1718115060000000000,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""


In [130]:
# Convert timestamp to datetime
option_df = option_df.with_columns(
    pl.from_epoch("timestamp", time_unit="ns").alias("timestamp")
)

option_df.head() 

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],str,str
"""O:A240621C00110000""",4,22.11,2024-06-11 14:11:00,"""A""",2024-06-21 00:00:00,"""C""","""00110000"""
"""O:A240621C00135000""",1,1.00,2024-06-11 13:30:00,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,2024-06-11 13:39:00,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""
"""O:A240621C00135000""",1,0.87,2024-06-11 13:40:00,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""
"""O:A240621C00135000""",1,0.85,2024-06-11 14:11:00,"""A""",2024-06-21 00:00:00,"""C""","""00135000"""


In [131]:
# Convert strike_price to decimal
option_df = option_df.with_columns(
    pl.col("strike_price").map_elements(lambda strike_price_str: float(strike_price_str) / 1000.0, return_dtype=pl.Float64).alias("strike_price")
)
option_df = option_df.with_columns(
    pl.col('strike_price').cast(pl.Decimal(scale=2)).alias('strike_price')
)

option_df.head()  

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],str,"decimal[*,2]"
"""O:A240621C00110000""",4,22.11,2024-06-11 14:11:00,"""A""",2024-06-21 00:00:00,"""C""",110.00
"""O:A240621C00135000""",1,1.00,2024-06-11 13:30:00,"""A""",2024-06-21 00:00:00,"""C""",135.00
"""O:A240621C00135000""",1,0.85,2024-06-11 13:39:00,"""A""",2024-06-21 00:00:00,"""C""",135.00
"""O:A240621C00135000""",1,0.87,2024-06-11 13:40:00,"""A""",2024-06-21 00:00:00,"""C""",135.00
"""O:A240621C00135000""",1,0.85,2024-06-11 14:11:00,"""A""",2024-06-21 00:00:00,"""C""",135.00


In [132]:
# Convert the option_type to use OptionType enum
from enum import Enum

class OptionType(Enum):
    CALL = "CALL"
    PUT = "PUT"

option_df = option_df.with_columns(
    pl.col("option_type").map_elements(
        lambda option_type_str: OptionType.CALL.value if option_type_str == "C" else OptionType.PUT.value, 
        return_dtype=pl.String
    ).cast(pl.Enum([OptionType.CALL.value, OptionType.PUT.value])).alias("option_type")
)

option_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,2]"
"""O:A240621C00110000""",4,22.11,2024-06-11 14:11:00,"""A""",2024-06-21 00:00:00,"""CALL""",110.00
"""O:A240621C00135000""",1,1.00,2024-06-11 13:30:00,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00135000""",1,0.85,2024-06-11 13:39:00,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00135000""",1,0.87,2024-06-11 13:40:00,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00135000""",1,0.85,2024-06-11 14:11:00,"""A""",2024-06-21 00:00:00,"""CALL""",135.00


In [133]:
# Compiled all the previous into a single function

def enhance_option_parquet(option_df: pl.DataFrame) -> pl.DataFrame:
    
    # Rename and select only required columns
    option_df = option_df.select('ticker', 'volume', 'open', 'window_start')
    option_df = option_df.rename({'ticker': 'option_ticker', 'open': 'option_price', 'window_start': 'timestamp'})
    
    # Parse Polygon option tickers to useful columns
    regex_pattern = r"O:([A-Za-z]+)([0-9]+)([PC])([0-9]+)"
    option_df = option_df.with_columns(
        pl.col('option_ticker').str.extract(regex_pattern, 1).alias('underlying_ticker'),
        pl.col('option_ticker').str.extract(regex_pattern, 2).alias('expiration_date'),
        pl.col('option_ticker').str.extract(regex_pattern, 3).alias('option_type'),
        pl.col('option_ticker').str.extract(regex_pattern, 4).alias('strike_price'),
    )
    
    # Expiration date can sometimes include an extra int at the front (because Polygon idk) so we need to filter those out and transform to proper type
    option_df = option_df.with_columns(
        pl.when(pl.col('expiration_date').str.len_chars() == 7)
        .then(pl.col("expiration_date").str.slice(1))
        .otherwise(pl.col('expiration_date'))
        .alias('expiration_date')
    )
    
    # Convert column types
    option_df = option_df.with_columns(
        pl.col("expiration_date").map_elements(
            lambda date_str: f"20{date_str[0:2]}-{date_str[2:4]}-{date_str[4:]}",
            return_dtype=pl.self_dtype()
            ).str.to_datetime(time_unit="ns").alias("expiration_date"),
        pl.col('option_price').cast(pl.Decimal(scale=2)).alias('option_price'),
        pl.from_epoch("timestamp", time_unit="ns").alias("timestamp"),
        pl.col("strike_price").map_elements(lambda strike_price_str: float(strike_price_str) / 1000.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=2)).alias("strike_price"),
        pl.col("option_type").map_elements(
            lambda option_type_str: OptionType.CALL.value if option_type_str == "C" else OptionType.PUT.value, 
            return_dtype=pl.String
        ).cast(pl.Enum([OptionType.CALL.value, OptionType.PUT.value])).alias("option_type")
    )
    
    return option_df

In [134]:
# Try it out on a fresh read
fresh_option_df = pl.read_parquet("test_data/options_minute_aggs.parquet")
fresh_option_df.head()

ticker,volume,open,close,high,low,window_start,transactions
str,i64,f64,f64,f64,f64,i64,i64
"""O:A240621C00110000""",4,22.11,22.11,22.11,22.11,1718115060000000000,1
"""O:A240621C00135000""",1,1.0,1.0,1.0,1.0,1718112600000000000,1
"""O:A240621C00135000""",1,0.85,0.85,0.85,0.85,1718113140000000000,1
"""O:A240621C00135000""",1,0.87,0.87,0.87,0.87,1718113200000000000,1
"""O:A240621C00135000""",1,0.85,0.85,0.85,0.85,1718115060000000000,1


In [135]:
enhanced_option_df = enhance_option_parquet(fresh_option_df)
enhanced_option_df.head()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,2]"
"""O:A240621C00110000""",4,22.11,2024-06-11 14:11:00,"""A""",2024-06-21 00:00:00,"""CALL""",110.00
"""O:A240621C00135000""",1,1.00,2024-06-11 13:30:00,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00135000""",1,0.85,2024-06-11 13:39:00,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00135000""",1,0.87,2024-06-11 13:40:00,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00135000""",1,0.85,2024-06-11 14:11:00,"""A""",2024-06-21 00:00:00,"""CALL""",135.00


In [136]:
# Now trying with a day_aggs (should be the same)
options_day_df = pl.read_parquet("test_data/options_day_aggs.parquet")
options_day_df.head()

ticker,volume,open,close,high,low,window_start,transactions
str,i64,f64,f64,f64,f64,i64,i64
"""O:A240621C00110000""",4,22.11,22.11,22.11,22.11,1718078400000000000,1
"""O:A240621C00135000""",21,1.0,0.95,1.15,0.85,1718078400000000000,9
"""O:A240621C00140000""",8,0.15,0.13,0.25,0.13,1718078400000000000,5
"""O:A240621C00145000""",1,0.03,0.03,0.03,0.03,1718078400000000000,1
"""O:A240621C00160000""",2,0.01,0.05,0.05,0.01,1718078400000000000,2


In [137]:
options_day_df = enhance_option_parquet(options_day_df)
options_day_df.head()


option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
str,i64,"decimal[*,2]",datetime[ns],str,datetime[ns],enum,"decimal[*,2]"
"""O:A240621C00110000""",4,22.11,2024-06-11 04:00:00,"""A""",2024-06-21 00:00:00,"""CALL""",110.00
"""O:A240621C00135000""",21,1.00,2024-06-11 04:00:00,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00140000""",8,0.15,2024-06-11 04:00:00,"""A""",2024-06-21 00:00:00,"""CALL""",140.00
"""O:A240621C00145000""",1,0.03,2024-06-11 04:00:00,"""A""",2024-06-21 00:00:00,"""CALL""",145.00
"""O:A240621C00160000""",2,0.01,2024-06-11 04:00:00,"""A""",2024-06-21 00:00:00,"""CALL""",160.00


In [155]:
# Should probably check for nulls
options_day_df.null_count()

option_ticker,volume,option_price,timestamp,underlying_ticker,expiration_date,option_type,strike_price
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


In [138]:
# Going to look at the trades ones now, likely a little different
option_trades_df = pl.read_parquet("test_data/options_trades.parquet")
option_trades_df.head()

ticker,conditions,correction,exchange,price,sip_timestamp,size
str,f64,i64,i64,f64,i64,i64
"""O:A240621C00110000""",209.0,0,316,22.11,1718115086030000000,4
"""O:A240621C00135000""",209.0,0,302,1.0,1718112619815000000,1
"""O:A240621C00135000""",232.0,0,322,0.85,1718113199235000000,1
"""O:A240621C00135000""",233.0,0,302,0.87,1718113202886000000,1
"""O:A240621C00135000""",209.0,0,308,0.85,1718115084469000000,1


In [139]:
# Pretty much everything is the same, just have to adjust selected columns

def enhance_option_trades_parquet(option_df: pl.DataFrame) -> pl.DataFrame:
    
    # Rename and select only required columns
    option_df = option_df.select('ticker', 'price', 'sip_timestamp', 'size')
    option_df = option_df.rename({'ticker': 'option_ticker', 'price': 'option_price', 'sip_timestamp':'timestamp', 'size': 'trade_size'})
    
    # Parse Polygon option tickers to useful columns
    regex_pattern = r"O:([A-Za-z]+)([0-9]+)([PC])([0-9]+)"
    option_df = option_df.with_columns(
        pl.col('option_ticker').str.extract(regex_pattern, 1).alias('underlying_ticker'),
        pl.col('option_ticker').str.extract(regex_pattern, 2).alias('expiration_date'),
        pl.col('option_ticker').str.extract(regex_pattern, 3).alias('option_type'),
        pl.col('option_ticker').str.extract(regex_pattern, 4).alias('strike_price'),
    )
    
    # Expiration date can sometimes include an extra int at the front (because Polygon idk) so we need to filter those out and transform to proper type
    option_df = option_df.with_columns(
        pl.when(pl.col('expiration_date').str.len_chars() == 7)
        .then(pl.col("expiration_date").str.slice(1))
        .otherwise(pl.col('expiration_date'))
        .alias('expiration_date')
    )
    
    # Convert column types
    option_df = option_df.with_columns(
        pl.col("expiration_date").map_elements(
            lambda date_str: f"20{date_str[0:2]}-{date_str[2:4]}-{date_str[4:]}",
            return_dtype=pl.self_dtype()
            ).str.to_datetime(time_unit="ns").alias("expiration_date"),
        pl.col('option_price').cast(pl.Decimal(scale=2)).alias('option_price'),
        pl.from_epoch("timestamp", time_unit="ns").alias("timestamp"),
        pl.col("strike_price").map_elements(lambda strike_price_str: float(strike_price_str) / 1000.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=2)).alias("strike_price"),
        pl.col("option_type").map_elements(
            lambda option_type_str: OptionType.CALL.value if option_type_str == "C" else OptionType.PUT.value, 
            return_dtype=pl.String
        ).cast(pl.Enum([OptionType.CALL.value, OptionType.PUT.value])).alias("option_type")
    )
    
    return option_df

option_trades_df = enhance_option_trades_parquet(option_trades_df)
option_trades_df.head()

option_ticker,option_price,timestamp,trade_size,underlying_ticker,expiration_date,option_type,strike_price
str,"decimal[*,2]",datetime[ns],i64,str,datetime[ns],enum,"decimal[*,2]"
"""O:A240621C00110000""",22.11,2024-06-11 14:11:26.030,4,"""A""",2024-06-21 00:00:00,"""CALL""",110.00
"""O:A240621C00135000""",1.00,2024-06-11 13:30:19.815,1,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00135000""",0.85,2024-06-11 13:39:59.235,1,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00135000""",0.87,2024-06-11 13:40:02.886,1,"""A""",2024-06-21 00:00:00,"""CALL""",135.00
"""O:A240621C00135000""",0.85,2024-06-11 14:11:24.469,1,"""A""",2024-06-21 00:00:00,"""CALL""",135.00


In [154]:
# Should probably check for nulls
option_trades_df.null_count()

option_ticker,option_price,timestamp,trade_size,underlying_ticker,expiration_date,option_type,strike_price
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


In [140]:
# Now to take a look at the stock parquets

stocks_df = pl.read_parquet("test_data/stocks_minute_aggs.parquet")
stocks_df.head()

ticker,volume,open,close,high,low,window_start,transactions
str,i64,f64,f64,f64,f64,i64,i64
"""A""",11265,132.64,132.38,132.835,132.38,1718112600000000000,70
"""A""",436,132.405,132.405,132.405,132.405,1718112660000000000,25
"""A""",728,132.0891,132.0891,132.0891,132.0891,1718112720000000000,33
"""A""",2745,131.55,131.735,131.86,131.55,1718112900000000000,91
"""A""",690,131.735,131.735,131.92,131.735,1718112960000000000,17


In [141]:
# Using a lot of the same functionality we can slightly adjust to work on stock aggs

def enhance_stock_parquet(stocks_df: pl.DataFrame) -> pl.DataFrame:
    stocks_df = stocks_df.select('ticker', 'volume', 'open', 'window_start')
    stocks_df = stocks_df.rename({'open': 'price', 'window_start': 'timestamp'})
    stocks_df = stocks_df.with_columns(
        pl.col('price').cast(pl.Decimal(scale=2)).alias('price'),
        pl.from_epoch("timestamp", time_unit="ns").alias("timestamp"),
    )
    return stocks_df

stocks_df = enhance_stock_parquet(stocks_df)
stocks_df.head()

ticker,volume,price,timestamp
str,i64,"decimal[*,2]",datetime[ns]
"""A""",11265,132.63,2024-06-11 13:30:00
"""A""",436,132.40,2024-06-11 13:31:00
"""A""",728,132.08,2024-06-11 13:32:00
"""A""",2745,131.55,2024-06-11 13:35:00
"""A""",690,131.73,2024-06-11 13:36:00


In [142]:
# Try it with a day_aggs to
stock_day_aggr_df = pl.read_parquet("test_data/stocks_day_aggs.parquet")
stock_day_aggr_df = enhance_stock_parquet(stock_day_aggr_df)
stock_day_aggr_df.head()

ticker,volume,price,timestamp
str,i64,"decimal[*,2]",datetime[ns]
"""A""",2081563,132.63,2024-06-11 04:00:00
"""AA""",3520311,40.25,2024-06-11 04:00:00
"""AAA""",10550,25.08,2024-06-11 04:00:00
"""AAAU""",1173193,22.88,2024-06-11 04:00:00
"""AACG""",11808,0.82,2024-06-11 04:00:00


In [143]:
# Checkout stock trades (when you have the bandwidth to download)

In [144]:
# Need to do the treasury data as well
treasury_yields_df = pl.read_parquet('test_data/treasury_yields.parquet')
treasury_yields_df.head()

date,yield_1_month,yield_3_month,yield_6_month,yield_1_year,yield_2_year,yield_3_year,yield_5_year,yield_7_year,yield_10_year,yield_20_year,yield_30_year
str,f64,f64,null,f64,f64,null,f64,null,f64,null,f64
"""1962-01-02""",null,null,null,3.22,null,null,3.88,null,4.06,null,null
"""1962-01-03""",null,null,null,3.24,null,null,3.87,null,4.03,null,null
"""1962-01-04""",null,null,null,3.24,null,null,3.86,null,3.99,null,null
"""1962-01-05""",null,null,null,3.26,null,null,3.89,null,4.02,null,null
"""1962-01-08""",null,null,null,3.31,null,null,3.91,null,4.03,null,null


In [145]:
# Might be better to look at the tail since that is the data most used
treasury_yields_df.tail()

date,yield_1_month,yield_3_month,yield_6_month,yield_1_year,yield_2_year,yield_3_year,yield_5_year,yield_7_year,yield_10_year,yield_20_year,yield_30_year
str,f64,f64,null,f64,f64,null,f64,null,f64,null,f64
"""2025-08-08""",4.48,4.32,null,3.93,3.76,null,3.84,null,4.27,null,4.85
"""2025-08-11""",4.46,4.34,null,3.93,3.76,null,3.83,null,4.27,null,4.84
"""2025-08-12""",4.44,4.33,null,3.9,3.72,null,3.83,null,4.29,null,4.88
"""2025-08-13""",4.44,4.29,null,3.86,3.67,null,3.77,null,4.24,null,4.83
"""2025-08-14""",4.48,4.3,null,3.91,3.74,null,3.82,null,4.29,null,4.88


In [146]:
# null values stand out so we need to fill those, ideally with the last valued column, lets see if this works
before_fill_df = treasury_yields_df.null_count()
nulll_filled_df = treasury_yields_df.with_columns(
    pl.col("yield_6_month").fill_null(strategy="forward").alias("yield_6_month")
)
after_fill_df = nulll_filled_df.null_count()
before_fill_df == after_fill_df

date,yield_1_month,yield_3_month,yield_6_month,yield_1_year,yield_2_year,yield_3_year,yield_5_year,yield_7_year,yield_10_year,yield_20_year,yield_30_year
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
true,true,true,true,true,true,true,true,true,true,true,true


In [147]:
# I guess none for the 6 month? I'll apply to all to see what happens
before_fill_df = treasury_yields_df.null_count()
treasury_yields_df = treasury_yields_df.with_columns(
    pl.col("yield_1_month").fill_null(strategy="forward").alias("yield_1_month"),
    pl.col("yield_3_month").fill_null(strategy="forward").alias("yield_3_month"),
    pl.col("yield_6_month").fill_null(strategy="forward").alias("yield_6_month"),
    pl.col("yield_1_year").fill_null(strategy="forward").alias("yield_1_year"),
    pl.col("yield_2_year").fill_null(strategy="forward").alias("yield_2_year"),
    pl.col("yield_3_year").fill_null(strategy="forward").alias("yield_3_year"),
    pl.col("yield_5_year").fill_null(strategy="forward").alias("yield_5_year"),
    pl.col("yield_7_year").fill_null(strategy="forward").alias("yield_7_year"),
    pl.col("yield_10_year").fill_null(strategy="forward").alias("yield_10_year"),
    pl.col("yield_20_year").fill_null(strategy="forward").alias("yield_20_year"),
    pl.col("yield_30_year").fill_null(strategy="forward").alias("yield_30_year"),
)
after_fill_df = nulll_filled_df.null_count()
before_fill_df == after_fill_df

date,yield_1_month,yield_3_month,yield_6_month,yield_1_year,yield_2_year,yield_3_year,yield_5_year,yield_7_year,yield_10_year,yield_20_year,yield_30_year
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
true,true,true,true,true,true,true,true,true,true,true,true


In [148]:
# So there are no 6m, 3y, 7y, 20y so we'll just select the others out and give them better names
treasury_yields_df = treasury_yields_df.select(['date', 'yield_1_month', 'yield_3_month', 'yield_1_year', 'yield_2_year', 'yield_5_year', 'yield_10_year', 'yield_30_year'])
treasury_yields_df = treasury_yields_df.rename({'yield_1_month': '1m', 'yield_3_month': '3m', 'yield_1_year': '1y', 'yield_2_year': '2y', 'yield_5_year': '5y', 'yield_10_year': '10y', 'yield_30_year': '30y'})
treasury_yields_df.tail()

date,1m,3m,1y,2y,5y,10y,30y
str,f64,f64,f64,f64,f64,f64,f64
"""2025-08-08""",4.48,4.32,3.93,3.76,3.84,4.27,4.85
"""2025-08-11""",4.46,4.34,3.93,3.76,3.83,4.27,4.84
"""2025-08-12""",4.44,4.33,3.9,3.72,3.83,4.29,4.88
"""2025-08-13""",4.44,4.29,3.86,3.67,3.77,4.24,4.83
"""2025-08-14""",4.48,4.3,3.91,3.74,3.82,4.29,4.88


In [149]:
# Better, now to do some conversions
treasury_yields_df = treasury_yields_df.with_columns(
    pl.col('date').str.to_datetime(time_unit="ns").alias('date'),
    pl.col('1m').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("1m"),
    pl.col('3m').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("3m"),
    pl.col('1y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("1y"),
    pl.col('2y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("2y"),
    pl.col('5y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("5y"),
    pl.col('10y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("10y"),
    pl.col('30y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("30y"),
)
treasury_yields_df.tail()

date,1m,3m,1y,2y,5y,10y,30y
datetime[ns],"decimal[*,6]","decimal[*,6]","decimal[*,6]","decimal[*,6]","decimal[*,6]","decimal[*,6]","decimal[*,6]"
2025-08-08 00:00:00,0.044800,0.043200,0.039300,0.037599,0.038400,0.042699,0.048499
2025-08-11 00:00:00,0.044600,0.043400,0.039300,0.037599,0.038300,0.042699,0.048400
2025-08-12 00:00:00,0.044400,0.043300,0.039000,0.037200,0.038300,0.042900,0.048799
2025-08-13 00:00:00,0.044400,0.042900,0.038599,0.036700,0.037700,0.042400,0.048300
2025-08-14 00:00:00,0.044800,0.043000,0.039100,0.037400,0.038200,0.042900,0.048799


In [150]:
# Check for nulls real quick
treasury_yields_df.null_count()

date,1m,3m,1y,2y,5y,10y,30y
u32,u32,u32,u32,u32,u32,u32,u32
0,9878,4901,0,3592,0,0,3770


In [151]:
# Not good, going to try to fill those with just the median value for dev work
treasury_yields_df = treasury_yields_df.with_columns(
    pl.col('1m').fill_null(pl.col('1m').median()).alias("1m"),
    pl.col('3m').fill_null(pl.col('3m').median()).alias("3m"),
    pl.col('1y').fill_null(pl.col('1y').median()).alias("1y"),
    pl.col('2y').fill_null(pl.col('2y').median()).alias("2y"),
    pl.col('5y').fill_null(pl.col('5y').median()).alias("5y"),
    pl.col('10y').fill_null(pl.col('10y').median()).alias("10y"),
    pl.col('30y').fill_null(pl.col('30y').median()).alias("30y")
)
treasury_yields_df.null_count()

date,1m,3m,1y,2y,5y,10y,30y
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


In [152]:
# Putting it all together
def enhance_treasury_parquet(treasury_yields_df: pl.DataFrame) -> pl.DataFrame:
    treasury_yields_df = treasury_yields_df.select(['date', 'yield_1_month', 'yield_3_month', 'yield_1_year', 'yield_2_year', 'yield_5_year', 'yield_10_year', 'yield_30_year'])
    treasury_yields_df = treasury_yields_df.rename({'yield_1_month': '1m', 'yield_3_month': '3m', 'yield_1_year': '1y', 'yield_2_year': '2y', 'yield_5_year': '5y', 'yield_10_year': '10y', 'yield_30_year': '30y'})
    treasury_yields_df = treasury_yields_df.with_columns(
        pl.col('date').str.to_datetime(time_unit="ns").alias('date'),
        pl.col('1m').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("1m"),
        pl.col('3m').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("3m"),
        pl.col('1y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("1y"),
        pl.col('2y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("2y"),
        pl.col('5y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("5y"),
        pl.col('10y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("10y"),
        pl.col('30y').map_elements(lambda ir: float(ir) / 100.0, return_dtype=pl.Float64).cast(pl.Decimal(scale=6)).alias("30y"),
    )
    treasury_yields_df = treasury_yields_df.with_columns(
        pl.col('1m').fill_null(pl.col('1m').median()).alias("1m"),
        pl.col('3m').fill_null(pl.col('3m').median()).alias("3m"),
        pl.col('1y').fill_null(pl.col('1y').median()).alias("1y"),
        pl.col('2y').fill_null(pl.col('2y').median()).alias("2y"),
        pl.col('5y').fill_null(pl.col('5y').median()).alias("5y"),
        pl.col('10y').fill_null(pl.col('10y').median()).alias("10y"),
        pl.col('30y').fill_null(pl.col('30y').median()).alias("30y")
    )
    return treasury_yields_df

enhanced_treasury_df = enhance_treasury_parquet(pl.read_parquet('test_data/treasury_yields.parquet'))
enhanced_treasury_df.tail() 

date,1m,3m,1y,2y,5y,10y,30y
datetime[ns],f64,f64,f64,f64,f64,f64,f64
2025-08-08 00:00:00,0.0448,0.0432,0.0393,0.037599,0.0384,0.042699,0.048499
2025-08-11 00:00:00,0.0446,0.0434,0.0393,0.037599,0.0383,0.042699,0.0484
2025-08-12 00:00:00,0.0444,0.0433,0.039,0.0372,0.0383,0.0429,0.048799
2025-08-13 00:00:00,0.0444,0.0429,0.038599,0.0367,0.0377,0.0424,0.0483
2025-08-14 00:00:00,0.0448,0.043,0.0391,0.0374,0.0382,0.0429,0.048799


In [153]:
# null check
enhanced_treasury_df.null_count()

date,1m,3m,1y,2y,5y,10y,30y
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0
